In [1]:
from bs4 import BeautifulSoup
import requests

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

webdriver.Chrome(ChromeDriverManager().install())


In [2]:


drivers = []
for i in range(5):
    drivers.append(webdriver.Chrome(ChromeDriverManager().install()))


C:\Users\pqhuy\AppData\Local\Temp\ipykernel_18748\38767446.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  drivers.append(webdriver.Chrome(ChromeDriverManager().install()))


In [3]:
def get_links(start, end):
    links = []
    for i in range(start, end):
        url = f'https://www.nhatot.com/mua-ban-bat-dong-san?page={i}'
        text = requests.get(url).text
        soup = BeautifulSoup(text, 'html5lib')
        for link in soup.find_all('a', attrs={'class': 'AdItem_adItem__2O28x'}):
            links.append(link['href'])
    return links

In [4]:
list_data = []
def get_data(driver, start, end):
    links = get_links(start, end)
    for link in links: 
        driver.get('https://nhatot.com' + link)
        dic = {}
        soup = BeautifulSoup(driver.page_source)
        for div in soup.find_all('div', attrs={'class': 'AdParam_adParamItem__1o-dd'}):
            att = div.span.span.getText()[:-2]
            value = div.span.find('span', attrs={'class': 'AdParam_adParamValue__1ayWO'}).getText()
            dic[att] = value
        dic['Giá'] = soup.find('span', attrs={'itemprop' : 'price'}).find(text=True)
        list_data.append(dic)

In [7]:
from threading import Thread

class myThread(Thread):
    def __init__(self, driver, page_start, page_end):
        Thread.__init__(self)
        self.driver = driver
        self.page_start = page_start
        self.page_end = page_end
    
    def run(self):
        get_data(self.driver, self.page_start, self.page_end)

for i in range(0, 5):
    thread = myThread(drivers[i], 51+i*10, 60+i*10)
    thread.start()

In [8]:
list_data

[{'Diện tích đất': '60 m²',
  'Giá/m2': '225 triệu/m²',
  'Số phòng ngủ': '10 phòng',
  'Hướng cửa chính': 'Nam',
  'Số phòng vệ sinh': 'Nhiều hơn 6 phòng',
  'Tổng số tầng': '5',
  'Giấy tờ pháp lý': 'Đã có sổ',
  'Loại hình nhà ở': 'Nhà ngõ',
  'Tình trạng nội thất': 'Nội thất đầy đủ',
  'Chiều ngang': '4 m',
  'Chiều dài': '15 m',
  'Diện tích sử dụng': '300 m²',
  'Giá': '13,5 tỷ'},
 {'Dự Án': 'Quy Nhơn MelodyNhấn để xem thông tin về dự án được sàng lọc uy tín và sát sao nhất thị trường',
  'Tình trạng bất động sản': 'Chưa bàn giao',
  'Diện tích': '50 m²',
  'Giá/m2': '32 triệu/m²',
  'Số phòng ngủ': '1 phòng',
  'Hướng cửa chính': 'Tây Bắc',
  'Số phòng vệ sinh': '1 phòng',
  'Hướng ban công': 'Đông Nam',
  'Loại hình căn hộ': 'Chung cư',
  'Giấy tờ pháp lý': 'Giấy tờ khác',
  'Tình trạng nội thất': 'Nội thất cao cấp',
  'Giá': '1,6 tỷ'},
 {'Dự Án': 'Đang cập nhật',
  'Diện tích đất': '110 m2',
  'Giá/m2': '5,09 triệu/m2',
  'Hướng cửa chính': 'Đông Nam',
  'Tên phân khu/Lô/Block

In [9]:
import pandas as pd
data = pd.DataFrame(list_data)
data

,Diện tích đất,Giá/m2,Số phòng ngủ,Hướng cửa chính,Số phòng vệ sinh,Tổng số tầng,Giấy tờ pháp lý,Loại hình nhà ở,Tình trạng nội thất,Chiều ngang,...,Hướng ban công,Loại hình căn hộ,Tên phân khu/Lô/Block/Tháp,Mã lô,Loại hình đất,Tầng số,Đặc điểm nhà/đất,Loại hình văn phòng,Đặc điểm căn hộ,Mã căn / Mã căn hộ
0,60 m²,225 triệu/m²,10 phòng,Nam,Nhiều hơn 6 phòng,5,Đã có sổ,Nhà ngõ,Nội thất đầy đủ,4 m,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,32 triệu/m²,1 phòng,Tây Bắc,1 phòng,NaN,Giấy tờ khác,NaN,Nội thất cao cấp,NaN,...,Đông Nam,Chung cư,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110 m2,"5,09 triệu/m2",NaN,Đông Nam,NaN,NaN,Đã có sổ,NaN,NaN,5 m,...,NaN,NaN,Hòa lạc trung tâm chợ Gạo Tiền Giang,19,Đất thổ cư,NaN,NaN,NaN,NaN,NaN
3,87 m²,"45,98 triệu/m²",NaN,Đông Bắc,NaN,NaN,Đã có sổ,NaN,NaN,4 m,...,NaN,NaN,NaN,NaN,Đất thổ cư,NaN,NaN,NaN,NaN,NaN
4,NaN,"30,71 triệu/m²",2 phòng,NaN,2 phòng,NaN,Đang chờ sổ,NaN,NaN,NaN,...,NaN,Chung cư,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,52 m²,"41,35 triệu/m²",2 phòng,NaN,2 phòng,1,Đã có sổ,Nhà ngõ,Nội thất đầy đủ,4 m,...,NaN,NaN,NaN,NaN,NaN,NaN,Hẻm xe hơi,NaN,NaN,NaN
526,NaN,"50,38 triệu/m²",1 phòng,NaN,1 phòng,NaN,NaN,NaN,Nội thất đầy đủ,NaN,...,NaN,Chung cư,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527,1200 m²,"2,25 triệu/m²",NaN,NaN,NaN,NaN,Đã có sổ,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Đất nông nghiệp,NaN,NaN,NaN,NaN,NaN
528,NaN,"25,36 triệu/m²",4 phòng,Đông Nam,2 phòng,NaN,Đã có sổ,NaN,NaN,NaN,...,NaN,Chung cư,NaN,NaN,NaN,NaN,NaN,NaN,Căn góc,NaN


In [13]:
data.to_csv('gia_chung_cu2.csv')


In [12]:
data.isnull().sum()/ len(data)

Diện tích đất                 0.130189
Giá/m2                        0.000000
Số phòng ngủ                  0.337736
Hướng cửa chính               0.686792
Số phòng vệ sinh              0.515094
Tổng số tầng                  0.654717
Giấy tờ pháp lý               0.249057
Loại hình nhà ở               0.460377
Tình trạng nội thất           0.660377
Chiều ngang                   0.326415
Chiều dài                     0.350943
Diện tích sử dụng             0.641509
Giá                           0.000000
Dự Án                         0.833962
Tình trạng bất động sản       0.879245
Diện tích                     0.869811
Hướng ban công                0.956604
Loại hình căn hộ              0.879245
Tên phân khu/Lô/Block/Tháp    0.939623
Mã lô                         0.990566
Loại hình đất                 0.671698
Tầng số                       0.971698
Đặc điểm nhà/đất              0.762264
Loại hình văn phòng           0.990566
Đặc điểm căn hộ               0.975472
Mã căn / Mã căn hộ       

In [37]:
data1 = pd.read_csv('gia_chung_cu.csv')
data2 = pd.read_csv('gia_chung_cu2.csv')
data = pd.concat([data1, data2], ignore_index=True)
attrs = (data.isnull().sum()/ len(data) < 0.5).tolist()
names = data.columns[attrs]
data[names].drop(columns='Unnamed: 0')

,Giá/m2,Số phòng ngủ,Giá,Diện tích đất,Giấy tờ pháp lý,Chiều ngang,Chiều dài,Loại hình nhà ở,Số phòng vệ sinh
0,31 triệu/m²,2 phòng,"2,48 tỷ",NaN,NaN,NaN,NaN,NaN,NaN
1,5 triệu/m²,NaN,"1,5 tỷ",300 m²,Đã có sổ,10 m,30 m,NaN,NaN
2,"22,58 triệu/m²",4 phòng,7 tỷ,310 m²,Đã có sổ,4.5 m,22 m,Nhà mặt phố,NaN
3,"134,92 triệu/m²",6 phòng,17 tỷ,126 m²,Đã có sổ,7 m,18 m,Nhà biệt thự,6 phòng
4,"32,33 triệu/m2",NaN,"4,85 tỷ",150 m2,NaN,5 m,30 m,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1515,"41,35 triệu/m²",2 phòng,"2,15 tỷ",52 m²,Đã có sổ,4 m,13 m,Nhà ngõ,2 phòng
1516,"50,38 triệu/m²",1 phòng,"2,67 tỷ",NaN,NaN,NaN,NaN,NaN,1 phòng
1517,"2,25 triệu/m²",NaN,"2,7 tỷ",1200 m²,Đã có sổ,NaN,NaN,NaN,NaN
1518,"25,36 triệu/m²",4 phòng,"1,75 tỷ",NaN,Đã có sổ,NaN,NaN,NaN,2 phòng
